# TORONTO

## Import Data

In [1]:
import numpy as np

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

In [2]:
webdata=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0) #read all URL tables
data_t=pd.DataFrame(webdata[0]) #select 1st table and convert to dataframe
data_t.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
data_t.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Replace Neighbourhood name by Borough name, if a cell has a borough but a Not assigned neighborhood

In [3]:
length_data=len(data_t['Postcode'])-1 #dataframe length
for i in range (0, length_data):
    if data_t['Neighborhood'].iloc[i]=='Not assigned':
        data_t['Neighborhood'].iloc[i]=data_t['Borough'].iloc[i]

### Remove remaining rows where Borough and neighbourhood are 'Not assigned'

In [4]:
data_t = data_t[data_t['Neighborhood']!='Not assigned'] #remove Neighborhood=Not assigned

In [5]:
data_t.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Merge Neighbourhood information from rows with same Postcode

In [6]:
data_t_drop_lines=[] #index of redundant lines to be removed after grouping postcodes
length_data=len(data_t['Postcode'])-1 #dataframe length

for i in range(0,length_data): #join info from cells with same postcode
    if data_t['Postcode'].iloc[i]==data_t['Postcode'].iloc[i+1]:
        data_t['Neighborhood'].iloc[i]=data_t['Neighborhood'].iloc[i] + ', ' + data_t['Neighborhood'].iloc[i+1]
        data_t_drop_lines.append(i+1)
        
data_t.drop(data_t.index[data_t_drop_lines], axis=0, inplace=True) #drop redundat lines from cells with same postcode

### SHAPE

In [7]:
data_t.shape

(103, 3)

### READ LAT/LONG

In [8]:
lat_long=pd.read_csv('http://cocl.us/Geospatial_data')
lat_long.rename(columns={'Postal Code':'Postcode'}, inplace=True)

In [9]:
final = data_t.merge(lat_long, on='Postcode')

In [10]:
final.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# FETCH GEOLOCATION DATA

In [11]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\edbpbe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [12]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final['Latitude'], final['Longitude'], final['Borough'], final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### select only neighbourhoods in 'Downtown Toronto' in name

In [13]:
toronto_final = final[final['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_final.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


### get coordinates from Dowtown Toronto

In [14]:
address = 'Downtown Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\edbpbe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


### visualize neighborhoods in Downtown Toronto

In [15]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

### Define Foursquare credentials and version

In [16]:
CLIENT_ID = 'DRARNP4SUW2J1FRUWIFRNXCIBIESD2SAPM5FQNKLYS3IWSJE' # your Foursquare ID
CLIENT_SECRET = 'U4H0PDTCYDQQKJUNJV2DIFFMTBKGEG045W1YU3QGANNN5Q4I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DRARNP4SUW2J1FRUWIFRNXCIBIESD2SAPM5FQNKLYS3IWSJE
CLIENT_SECRET:U4H0PDTCYDQQKJUNJV2DIFFMTBKGEG045W1YU3QGANNN5Q4I


#### choose 1st neighborhood for analysis

In [17]:
toronto_final.loc[0, 'Neighborhood']

'Harbourfront, Regent Park'

#### Get the neighborhood's latitude and longitude values.

In [18]:
neighborhood_latitude = toronto_final.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_final.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_final.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront, Regent Park are 43.6542599, -79.3606359.


#### Now, let's get the top 10 venues that are in Harbourfront, Regent Park within a radius of 300 meters

#### create Foursquare URL

In [19]:
# type your answer here
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 300 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DRARNP4SUW2J1FRUWIFRNXCIBIESD2SAPM5FQNKLYS3IWSJE&client_secret=U4H0PDTCYDQQKJUNJV2DIFFMTBKGEG045W1YU3QGANNN5Q4I&v=20180605&ll=43.6542599,-79.3606359&radius=300&limit=50'

#### fetch information from Foursquare

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c36556add57970a2f67c0e5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 43.6569599027, 'lng': -79.35691110008916},
   'sw': {'lat': 43.6515598973, 'lng': -79.36436069991085}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',


In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Clean the json and structure it into a pandas dataframe.

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

12 venues were returned by Foursquare.


## EXPAND TO ALL NEIGHBORHOODS IN TORONTO

Function to repeat process to all the neighborhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### code to run the above function on each neighborhood and create a new dataframe called toronto_venues

In [25]:
toronto_venues = getNearbyVenues(names=toronto_final['Neighborhood'],
                                   latitudes=toronto_final['Latitude'],
                                   longitudes=toronto_final['Longitude']
                                  )


Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King
Harbourfront East, Toronto Islands
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park
CN Tower, Bathurst Quay
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


In [26]:
print(toronto_venues.shape)
toronto_venues.head()

(766, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


#### Let's check how many venues were returned for each neighborhood

In [27]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"CN Tower, Bathurst Quay",14,14,14,14,14,14
"Cabbagetown, St. James Town",49,49,49,49,49,49
Central Bay Street,50,50,50,50,50,50
"Chinatown, Grange Park",50,50,50,50,50,50
Christie,16,16,16,16,16,16
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50


#### Let's find out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 178 uniques categories.


# Neighborhood Analysis

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [30]:
toronto_onehot.shape

(766, 178)

#### neighborhood grouping by rown, by taking the mean of the frequency of occurrence of each category

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.04,0.0000,0.00,0.0000,0.00,0.000000,0.000000,0.040000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.02,0.040000,0.000000,0.00,0.02,0.02,0.000000,0.060000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.02,0.040000,0.000000,0.000000,0.00,0.000000,0.00,0.02,0.000000,0.00,0.00,0.00,0.020000,0.02,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.02,0.00,0.00,0.000000,0.00,0.00,0.040000,0.00,0.000000,0.02,0.000000,0.00,0.00,0.020000,0.0000,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.040000,0.00,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.00,0.00,0.000000,0.02,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.000000,0.020000,0.000000,0.02,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.02,0.00,0.00,0.000000,0.000000,0.00,0.02,0.040000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.02,0.000000,0.00,0.00,0.02,0.00,0.000000,0.00,0.000000,0.02,0.00,0.00,0.06,0.00,0.020000,0.00,0.00,0.000000,0.00,0.020000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00,0.00
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.020000,0.00,0.00,0.00,0.0000,0.00,0.0000,0.02,0.040000,0.000000,0.000000,0.02,0.02,0.040000,0.000000,0.02,0.02,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.040000,0.000000,0.02,0.000000,0.000000,0.00

In [32]:
toronto_grouped.shape

(18, 178)

#### Let's print each neighborhood along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King----
                 venue  freq
0                 Café  0.06
1           Steakhouse  0.06
2  American Restaurant  0.06
3                Hotel  0.04
4     Asian Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1    Restaurant  0.06
2  Cocktail Bar  0.06
3        Bakery  0.04
4          Café  0.04


----CN Tower, Bathurst Quay----
              venue  freq
0  Airport Terminal  0.14
1    Airport Lounge  0.14
2   Airport Service  0.14
3             Plane  0.07
4     Boat or Ferry  0.07


----Cabbagetown, St. James Town----
         venue  freq
0   Restaurant  0.08
1  Coffee Shop  0.08
2  Pizza Place  0.06
3          Pub  0.04
4         Café  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.08
2     Thai Restaurant  0.04
3  Chinese Restaurant  0.04
4     Bubble Tea Shop  0.04


----Chinatown, Grange Park----
                           venue  freq
0                         

#### Create dataframe with top 10 most common venues

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King",American Restaurant,Steakhouse,Café,Hotel,Bar,Asian Restaurant,Restaurant,Breakfast Spot,Coffee Shop,Gastropub
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Farmers Market,Pub,Beer Bar,Bakery,Steakhouse,Café,Seafood Restaurant
2,"CN Tower, Bathurst Quay",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pizza Place,Bakery,Café,Park,Italian Restaurant,Pub,Breakfast Spot,Deli / Bodega
4,Central Bay Street,Coffee Shop,Café,Bubble Tea Shop,Burger Joint,Falafel Restaurant,Spa,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Bar
5,"Chinatown, Grange Park",Café,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Bar,Bakery,Caribbean Restaurant,Farmers Market,Organic Grocery
6,Christie,Grocery Store,Café,Park,Nightclub,Diner,Baby Store,Athletics & Sports,Italian Restaurant,Convenience Store,Restaurant
7,Church and Wellesley,Coffee Shop,Burger Joint,Japanese Restaurant,Men's Store,Gay Bar,Restaurant,Ramen Restaurant,Pub,Bookstore,Pizza Place
8,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Hotel,American Restaurant,Gastropub,Deli / Bodega,Restaurant,Gym,Seafood Restaurant,Thai Restaurant
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Restaurant,Deli / Bodega,Gastropub,Seafood Restaurant,Steakhouse,Soup Place,Concert Hall,Fried Chicken Joint


# CLUSTERIZATION

In [36]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 5, 1, 1, 4, 0, 4, 3, 3])

In [37]:
toronto_merged = toronto_final

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,3,Coffee Shop,Café,Bakery,Pub,Park,Mexican Restaurant,Theater,Breakfast Spot,Restaurant,Performing Arts Venue
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Café,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Ramen Restaurant,Restaurant,Theater,Diner,Shopping Mall,Seafood Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,5,Coffee Shop,Gastropub,Restaurant,Italian Restaurant,Hotel,Farmers Market,Cocktail Bar,BBQ Joint,Beer Bar,Japanese Restaurant
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Restaurant,Farmers Market,Pub,Beer Bar,Bakery,Steakhouse,Café,Seafood Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Café,Bubble Tea Shop,Burger Joint,Falafel Restaurant,Spa,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Bar


#### Cluster visualization

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# CLUSTER DESIGNATION

#### CLUSTER AMERICAN-ASIAN

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,0,American Restaurant,Steakhouse,Café,Hotel,Bar,Asian Restaurant,Restaurant,Breakfast Spot,Coffee Shop,Gastropub


#### CLUSTER COFFEE FAMILY

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Café,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Ramen Restaurant,Restaurant,Theater,Diner,Shopping Mall,Seafood Restaurant
3,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Restaurant,Farmers Market,Pub,Beer Bar,Bakery,Steakhouse,Café,Seafood Restaurant
4,Downtown Toronto,1,Coffee Shop,Café,Bubble Tea Shop,Burger Joint,Falafel Restaurant,Spa,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Bar
13,Downtown Toronto,1,Park,Playground,Trail,Wings Joint,Concert Hall,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio


#### CLUSTER COZY

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Café,Cocktail Bar,Pub,Restaurant,Hotel,Farmers Market,Creperie,Bakery,Seafood Restaurant,Thai Restaurant


CLUSTER HOTEL BUSINESS

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Café,Bakery,Pub,Park,Mexican Restaurant,Theater,Breakfast Spot,Restaurant,Performing Arts Venue
8,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Deli / Bodega,Gastropub,Seafood Restaurant,Steakhouse,Soup Place,Concert Hall,Fried Chicken Joint
9,Downtown Toronto,3,Café,Coffee Shop,Hotel,American Restaurant,Gastropub,Deli / Bodega,Restaurant,Gym,Seafood Restaurant,Thai Restaurant
10,Downtown Toronto,3,Café,Coffee Shop,Restaurant,Bookstore,Bakery,Bar,Japanese Restaurant,Jazz Club,Comfort Food Restaurant,College Gym
16,Downtown Toronto,3,Café,Coffee Shop,Hotel,Deli / Bodega,Restaurant,Gastropub,American Restaurant,Bar,Concert Hall,Steakhouse
17,Downtown Toronto,3,Coffee Shop,Burger Joint,Japanese Restaurant,Men's Store,Gay Bar,Restaurant,Ramen Restaurant,Pub,Bookstore,Pizza Place


#### CLUSTER AIRPORT LIKE

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,4,Grocery Store,Café,Park,Nightclub,Diner,Baby Store,Athletics & Sports,Italian Restaurant,Convenience Store,Restaurant
7,Downtown Toronto,4,Coffee Shop,Aquarium,Café,Brewery,Pizza Place,Hotel,Scenic Lookout,Park,Train Station,Bakery
11,Downtown Toronto,4,Café,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Bar,Bakery,Caribbean Restaurant,Farmers Market,Organic Grocery
12,Downtown Toronto,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina
15,Downtown Toronto,4,Coffee Shop,Restaurant,Pizza Place,Bakery,Café,Park,Italian Restaurant,Pub,Breakfast Spot,Deli / Bodega


#### CLUSTER CONVENIENCE

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,5,Coffee Shop,Gastropub,Restaurant,Italian Restaurant,Hotel,Farmers Market,Cocktail Bar,BBQ Joint,Beer Bar,Japanese Restaurant
